In [ ]:
## Config
MODEL_PATH="../resources/ckpts/lgb"
DATA_PATH="../resources/data/"

In [ ]:
## 初始化
import numpy as np
import backtrader as bt
import sys; sys.path.append("..")
from utils.models import *

#############################
## 回测
#############################
class MyPandasData(bt.feeds.PandasData):
    lines = ('predict',)
    params = (
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('predict', 'predict')
    )


class MyStrategy(bt.Strategy):
    '''
    根据过去20日的收盘价斜率来选择股票。
    如果持仓少于2个股票，且信号为正，则买入斜率最高的60%的股票，并设置止盈止损。
    如果持仓超过2个股票，且信号为负，则卖出股票。
    '''
    params=(
            ('stopup', 0.22), 
            ('stopdown', 0.08), 
            ('maperiod',15),
            ("lookback", 20),
            ('RSRS_period', 18),
            ('RSRS_avg_period', 600),
            )
    def __init__(self):
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buy_list = []
        self.portfolio_values = []  # 用于记录组合净值

    def downcast(self, amount, lot):
        return abs(amount//lot*lot)
    

    #策略核心，根据条件执行买卖交易指令（必选）
    def next(self):
        # 每个时间点记录当前组合价值
        self.portfolio_values.append(self.broker.getvalue())
        if self.order: # 检查是否有指令等待执行, 
            return
        ### 计算股票池的动量（过去20日收盘价斜率）
        flag = False
        slope_period = []
        for code in set(self.getdatanames()):
            data = self.getdatabyname(code)
            closes_period = data.close.get(ago=-1, size=20).tolist()
            if len(closes_period) >= 20:
                flag = True
                slope = np.polyfit(range(len(closes_period)), closes_period, 1)[0]
            else:
                # 若数据不足，赋值为无效斜率
                slope = -np.inf
            slope_period.append((code, slope))
        if flag:
            slope_period = [x for x in slope_period if x[1] != -np.inf]
            slope_period = sorted(slope_period, key=lambda x: x[1])
            slope_period = slope_period[::-1]
            # print(slope_period)
            trade_codes = [x[0] for x in slope_period[:int(len(slope_period)*0.6)]]
        else:
            trade_codes = self.getdatanames()
        ###
        if len(self.buy_list) < 2:
            for code in set(trade_codes) - set(self.buy_list):
            # for code in set(self.getdatanames()) - set(self.buy_list):
                data = self.getdatabyname(code)
                price = data.close[0]
                price_up = price*(1 + self.params.stopup) # 止盈价
                price_down = price*(1-self.params.stopdown) # 止损价
                if data.predict[0] > 0:
                    order_value = self.broker.getvalue()*0.33
                    order_amount = self.downcast(order_value/data.close[0], 100)
                    self.order = self.buy_bracket(data, size=order_amount, name=code, limitprice = price_up, stopprice = price_down, exectype = bt.Order.Market)
                    # self.order = self.buy(data, size=order_amount, name=code)
                    self.buy_list.append(code)
        else:
        # elif self.position:
            now_list = []
            for code in self.buy_list:
                data = self.getdatabyname(code)
                if data.predict[0] < 0:
                    self.order = self.order_target_percent(data, 0, name=code)
                    continue
                now_list.append(code)
            self.buy_list = now_list.copy()

# 载入模型
model = BaseModel.use_subclass('lgb')()
model.load(MODEL_PATH)

# 创建Cerebro并添加数据
cerebro = bt.Cerebro()
for instrument, group in model.pred(df_19, features).groupby('instrument'):
    group = group.reset_index(level=0, drop=True)
    cerebro.adddata(MyPandasData(dataname=group), name=instrument)
cerebro.addstrategy(MyStrategy)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="SharpeRatio")
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="AnnualReturn")
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="DrawDown")

cerebro.broker.set_cash(1e6) # 设定初始资金
cerebro.broker.setcommission(commission=6e-4) # 设定手续费


In [ ]:
## 回测

import matplotlib.pyplot as plt
%matplotlib inline

result = cerebro.run()
print(result[0].analyzers.SharpeRatio.get_analysis())
print(result[0].analyzers.AnnualReturn.get_analysis())
print(result[0].analyzers.DrawDown.get_analysis()['max'])

# 获取组合的净值曲线
portfolio_value = cerebro.broker.get_value()
portfolio_values = result[0].portfolio_values

# 绘制净值曲线
plt.figure(figsize=(12, 6))
plt.plot(portfolio_values, label='Portfolio Value')
plt.title('Portfolio Value Over Time')
plt.xlabel('Time')
plt.ylabel('Portfolio Value')
plt.legend()
plt.show()